In [1]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
! pip install openai
! pip install tiktoken

#Pinecone first

  Obtaining dependency information for pinecone-client from https://files.pythonhosted.org/packages/cb/bb/c51fa42d85f431b3b3ec4c35a13a8bb99cafc0671918139a48767421d354/pinecone_client-3.2.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 6.0 MB/s eta 0:00:00
  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/c9/d1/450b19bbdbb2c802f554312c62ce2a2c0d8744fe14735bc70ad2803578c7/pypdf-4.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.1 MB/s eta 0:00:00a 0:00:01


  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/9e/11/83ca4e19bb6fc15971e543725ae1269a8a1c133e55b5952801ab9c0bcc9e/tiktoken-0.6.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 13.5 MB/s eta 0:00:00 0:00:01


In [23]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [24]:
!mkdir pdfs

mkdir: pdfs: File exists


In [25]:
loader=PyPDFDirectoryLoader("pdfs")

In [26]:
data=loader.load()

In [27]:
data[0]

Document(page_content='IMM 5754 (09-2023) E GCMSUCI:Date:\nType ValidityDocument\nNumberDear \nYour application to remain in Canada as a temporary resident has been approved.  A secure\ndocument has been mailed to you at the address that you provided when you made your\napplication.  If you have not received your secure document within six weeks,                       .\nPlease note that as per the Immigration and Refugee Protection Regulations (IRPR), a valid\nmedical examination is required if you are seeking to work in certain occupations in which the\nprotection of public health is essential.  Please review the following page to find out more aboutsuch occupations:  \nIf you do not have valid medical results you may have received condition 15 or 16 printed on your\npermit stating “Not authorized to work in 1) child care, 2) primary or secondary school teaching, 3)\nhealth services field occupations ( condition 15 ), 4) agricultural occupations ( condition 16 )”. If\nyou wish to rem

In [28]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [29]:
text_chunks = text_splitter.split_documents(data)

In [30]:
text_chunks

[Document(page_content='IMM 5754 (09-2023) E GCMSUCI:Date:\nType ValidityDocument\nNumberDear \nYour application to remain in Canada as a temporary resident has been approved.  A secure\ndocument has been mailed to you at the address that you provided when you made your\napplication.  If you have not received your secure document within six weeks,                       .\nPlease note that as per the Immigration and Refugee Protection Regulations (IRPR), a valid', metadata={'source': 'pdfs/IMM5754_2-15LCS32Z.pdf', 'page': 0}),
 Document(page_content='medical examination is required if you are seeking to work in certain occupations in which the\nprotection of public health is essential.  Please review the following page to find out more aboutsuch occupations:  \nIf you do not have valid medical results you may have received condition 15 or 16 printed on your\npermit stating “Not authorized to work in 1) child care, 2) primary or secondary school teaching, 3)', metadata={'source': 'pdfs/I

In [31]:
len(text_chunks)

5

In [32]:
print(text_chunks[0].page_content)

IMM 5754 (09-2023) E GCMSUCI:Date:
Type ValidityDocument
NumberDear 
Your application to remain in Canada as a temporary resident has been approved.  A secure
document has been mailed to you at the address that you provided when you made your
application.  If you have not received your secure document within six weeks,                       .
Please note that as per the Immigration and Refugee Protection Regulations (IRPR), a valid


In [33]:
print(text_chunks[1].page_content)

medical examination is required if you are seeking to work in certain occupations in which the
protection of public health is essential.  Please review the following page to find out more aboutsuch occupations:  
If you do not have valid medical results you may have received condition 15 or 16 printed on your
permit stating “Not authorized to work in 1) child care, 2) primary or secondary school teaching, 3)


In [34]:
print(text_chunks[2].page_content)

health services field occupations ( condition 15 ), 4) agricultural occupations ( condition 16 )”. If
you wish to remove these conditions on your Permit, you will need to complete a medical
examination and then, you must submit a new application. You also need a valid medical exam if
you are coming to Canada for more than 6 months and you lived in one or more of these
                                   for at least 6 months in a row within the last year.


In [35]:
print(text_chunks[4].page_content)

Canada
VISHALKUMAR  BHAGA VATIBHAI  PATEL,
Permit  - WP-EXT / Permis  - PT-PROR
2027 /01/18
U517002541


In [ ]:
import os
os.environ["OPENAI_API_KEY"]="YOUR_API_KEY"

In [37]:
embedding=OpenAIEmbeddings()

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [38]:
len(embedding.embed_query("How are you"))

NameError: name 'embedding' is not defined

In [39]:
len(embedding.embed_query("hi i am fine"))

NameError: name 'embedding' is not defined

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'Your Own Key')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [41]:
import pinecone

In [42]:
pinecone.init( api_key=PINECONE_API_KEY,  # find at app.pinecone.io
     environment=PINECONE_API_ENV  # next to api key in console
)

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [32]:
index_name = "testing" # put in the name of your pinecone index here

In [33]:
index = pinecone.Index('testing')


## Create Embeddings for each of the Text Chunk

In [35]:
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [36]:
docsearch

In [44]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x7f1fa7300df0>)

In [37]:
query= "YOLOv7 outperforms which models"

In [38]:
docs = docsearch.similarity_search(query)

In [39]:
docs

[Document(page_content='YOLOv7-tiny 6.2 3.5 320 30.8% 47.3% 32.2% 10.0% 31.9% 52.2%\nimprovement -39% -49% - = = = -0.9 = +0.7\nYOLOR-E6 [81] 115.8M 683.2G 1280 55.7% 73.2% 60.7% 40.1% 60.4% 69.2%\nYOLOv7-E6 97.2M 515.2G 1280 55.9% 73.5% 61.1% 40.6% 60.3% 70.0%\nimprovement -19% -33% - +0.2 +0.3 +0.4 +0.5 -0.1 +0.8\nYOLOR-D6 [81] 151.7M 935.6G 1280 56.1% 73.9% 61.2% 42.4% 60.5% 69.9%\nYOLOv7-D6 154.7M 806.8G 1280 56.3% 73.8% 61.4% 41.3% 60.6% 70.1%\nYOLOv7-E6E 151.7M 843.2G 1280 56.8% 74.4% 62.1% 40.8% 62.1% 70.6%'),
 Document(page_content='YOLOv5-L6 (r6.1) [23] 76.8M 445.6G 1280 63 - / 53.7% - -\nYOLOX-X [21] 99.1M 281.9G 640 58 51.5% / 51.1% - -\nYOLOv7-E6 97.2M 515.2G 1280 56 56.0% /55.9% 73.5% 61.2%\nYOLOR-E6 [81] 115.8M 683.2G 1280 45 55.8% / 55.7% 73.4% 61.1%\nPPYOLOE-X [85] 98.4M 206.6G 640 45 52.2% / 51.9% 69.9% 56.5%\nYOLOv7-D6 154.7M 806.8G 1280 44 56.6% /56.3% 74.0% 61.8%\nYOLOv5-X6 (r6.1) [23] 140.7M 839.2G 1280 38 - / 55.0% - -\nYOLOv7-E6E 151.7M 843.2G 1280 36 56.8% /56.8

In [40]:
llm = OpenAI()

In [41]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


In [42]:
query = "YOLOv7 outperforms which models"

In [43]:
qa.run(query)

' YOLOv7 outperforms YOLOv5-L6, YOLOX-X, YOLOR-E6, PPYOLOE-X, YOLOv5-X6, and YOLOR-CSP.'

In [45]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt:  what is a yolo?


Answer:  YOLO (You Only Look Once) is a type of object detector, specifically a deep learning algorithm used for object detection in images and videos. It was developed by Chien-Yao Wang, Alexey Bochkovskiy, and Hong-Yuan Mark Liao from the Institute of Information Science at Academia Sinica in Taiwan. YOLOv7 is the latest version of the algorithm, which has set a new state-of-the-art for real-time object detectors in terms of speed and accuracy.


Input Prompt:  who is invented the yolo?


Answer:  Joseph Redmon and Ali Farhadi


Input Prompt:  what was the accuracy of the yolov7?


Answer:  The accuracy of the YOLOv7 was 56.8% AP test-dev / 56.8% AP min-val.


Input Prompt:  exit


Exiting


SystemExit: 

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
